In [68]:
import pandas as pd
import datetime as dt
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.width", 200)
pd.set_option("display.float_format", lambda x : "%.3f" % x)

Read data

In [69]:
df =pd.read_csv("flo_data_20k.csv")

Exploratary data

In [70]:
def check_df(dataframe):
    print("########### Head ############")
    print(dataframe.head())
    print("########### Info ############")
    print(dataframe.info())
    print("########### NA ############")
    print(dataframe.isnull().sum())
    print("########### Tail ############")
    print(dataframe.tail())
    print("########### Describe ############")
    print(dataframe.describe().T)

In [71]:
check_df(df)

########### Head ############
                              master_id order_channel last_order_channel first_order_date last_order_date last_order_date_online last_order_date_offline  order_num_total_ever_online  \
0  cc294636-19f0-11eb-8d74-000d3a38a36f   Android App            Offline       2020-10-30      2021-02-26             2021-02-21              2021-02-26                        4.000   
1  f431bd5a-ab7b-11e9-a2fc-000d3a38a36f   Android App             Mobile       2017-02-08      2021-02-16             2021-02-16              2020-01-10                       19.000   
2  69b69676-1a40-11ea-941b-000d3a38a36f   Android App        Android App       2019-11-27      2020-11-27             2020-11-27              2019-12-01                        3.000   
3  1854e56c-491f-11eb-806e-000d3a38a36f   Android App        Android App       2021-01-06      2021-01-17             2021-01-17              2021-01-06                        1.000   
4  d6ea1074-f1f5-11e9-9346-000d3a38a36f      

# Outliers

Obtain outlier threshold

In [72]:
def outlier_threshod(dataframe,variable, q1=0.01, q3=0.99):
    quartile_1 = dataframe[variable].quantile(q1)
    quartile_3 = dataframe[variable].quantile(q3)
    IQR = quartile_3 - quartile_1
    up_limit = quartile_3 + 1.5 * IQR
    low_limit = quartile_1 - 1.5 * IQR
    return up_limit, low_limit

Trimming outliers

In [73]:
def replace_with_threshold(dataframe, variable):
    up_limit, low_limit =outlier_threshod(dataframe, variable)
    dataframe.loc[(dataframe[variable]< low_limit, variable)] = low_limit
    dataframe.loc[(dataframe[variable]> up_limit, variable)] = up_limit


In [74]:
num_cols = [col for col in df.columns if df[col].dtypes != "O"]

In [75]:
num_cols

['order_num_total_ever_online',
 'order_num_total_ever_offline',
 'customer_value_total_ever_offline',
 'customer_value_total_ever_online']

In [76]:
for col in num_cols:
    replace_with_threshold(df,col)

Total number of order

In [77]:
df["total_num_order"] = df["order_num_total_ever_offline"]+df["order_num_total_ever_online"]

Total value of customer

In [78]:
df["total_value_customer"] = df["customer_value_total_ever_offline"] + df["customer_value_total_ever_online"]

Check date variables

In [79]:
df.columns.str.contains("date").any()

True

In [80]:
date_columns = df.columns[df.columns.str.contains("date")]

In [81]:
date_columns

Index(['first_order_date', 'last_order_date', 'last_order_date_online', 'last_order_date_offline'], dtype='object')

In [82]:
df[date_columns] = df[date_columns].apply(pd.to_datetime)

In [83]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19945 entries, 0 to 19944
Data columns (total 14 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   master_id                          19945 non-null  object        
 1   order_channel                      19945 non-null  object        
 2   last_order_channel                 19945 non-null  object        
 3   first_order_date                   19945 non-null  datetime64[ns]
 4   last_order_date                    19945 non-null  datetime64[ns]
 5   last_order_date_online             19945 non-null  datetime64[ns]
 6   last_order_date_offline            19945 non-null  datetime64[ns]
 7   order_num_total_ever_online        19945 non-null  float64       
 8   order_num_total_ever_offline       19945 non-null  float64       
 9   customer_value_total_ever_offline  19945 non-null  float64       
 10  customer_value_total_ever_online  

## Create CLTV dataframe

In [84]:
df["last_order_date"].max()

Timestamp('2021-05-30 00:00:00')

In [85]:
analysis_data = dt.datetime(2021,6,1)

In [86]:
cltv_df = pd.DataFrame()

In [87]:
cltv_df["customer_id"] = df["master_id"]

In [88]:
cltv_df["recency_cltv_weekly"] = ((df["last_order_date"] - df["first_order_date"]).dt.days) / 7


In [90]:
cltv_df["T_weekly"] = ((analysis_data - df["first_order_date"]).dt.days) / 7

In [91]:
cltv_df["frequency"] = df["total_num_order"]

In [92]:
cltv_df["monetary_cltv_avg"] = df["total_value_customer"] / df["total_num_order"]

In [93]:
cltv_df.head()

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.000,30.571,5.000,187.874
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.857,224.857,21.000,95.883
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.286,78.857,5.000,117.064
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.571,20.857,2.000,60.985
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.143,95.429,2.000,104.990


# Create BG/NBD model

In [94]:
bgf = BetaGeoFitter(penalizer_coef=0.001)

In [95]:
bgf.fit(cltv_df["frequency"], 
        cltv_df["recency_cltv_weekly"],
        cltv_df["T_weekly"])

<lifetimes.BetaGeoFitter: fitted with 19945 subjects, a: 0.00, alpha: 76.17, b: 0.00, r: 3.66>

Predict expected purchase in 3 months 

In [96]:
cltv_df["exp_sales_3_months"] = bgf.predict(4+3,
                                            cltv_df["frequency"],
                                            cltv_df["recency_cltv_weekly"],
                                            cltv_df["T_weekly"])

Predict expected purchase in 6 months

In [97]:
cltv_df["exp_sales_6_months"] = bgf.predict(4+6,
                                            cltv_df["frequency"],
                                            cltv_df["recency_cltv_weekly"],
                                            cltv_df["T_weekly"])

In [98]:
cltv_df.head()

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_months,exp_sales_6_months
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.000,30.571,5.000,187.874,0.568,0.812
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.857,224.857,21.000,95.883,0.574,0.819
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.286,78.857,5.000,117.064,0.391,0.559
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.571,20.857,2.000,60.985,0.409,0.584
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.143,95.429,2.000,104.990,0.231,0.330


In [99]:
cltv_df.sort_values("exp_sales_3_months", ascending=False)[:10]

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_months,exp_sales_6_months
7330,a4d534a2-5b1b-11eb-8dbd-000d3a38a36f,62.714,67.286,52.500,164.633,2.740,3.915
15611,4a7e875e-e6ce-11ea-8f44-000d3a38a36f,39.714,40.000,29.000,165.298,1.968,2.812
8328,1902bf80-0035-11eb-8341-000d3a38a36f,28.857,33.286,25.000,97.440,1.833,2.619
19538,55d54d9e-8ac7-11ea-8ec0-000d3a38a36f,52.571,58.714,31.000,228.530,1.799,2.570
14373,f00ad516-c4f4-11ea-98f7-000d3a38a36f,38.000,46.429,27.000,141.355,1.751,2.501
10489,7af5cd16-b100-11e9-9757-000d3a38a36f,103.143,111.857,43.000,157.113,1.737,2.482
4315,d5ef8058-a5c6-11e9-a2fc-000d3a38a36f,133.143,147.143,49.500,160.203,1.666,2.381
6756,27310582-6362-11ea-a6dc-000d3a38a36f,62.714,64.143,29.000,168.881,1.630,2.328
6666,53fe00d4-7b7a-11eb-960b-000d3a38a36f,9.714,13.000,17.000,259.865,1.622,2.317
10536,e143b6fa-d6f8-11e9-93bc-000d3a38a36f,104.571,113.429,40.000,176.200,1.612,2.303


In [100]:
cltv_df.sort_values("exp_sales_6_months", ascending=False)[:10]

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_months,exp_sales_6_months
7330,a4d534a2-5b1b-11eb-8dbd-000d3a38a36f,62.714,67.286,52.500,164.633,2.740,3.915
15611,4a7e875e-e6ce-11ea-8f44-000d3a38a36f,39.714,40.000,29.000,165.298,1.968,2.812
8328,1902bf80-0035-11eb-8341-000d3a38a36f,28.857,33.286,25.000,97.440,1.833,2.619
19538,55d54d9e-8ac7-11ea-8ec0-000d3a38a36f,52.571,58.714,31.000,228.530,1.799,2.570
14373,f00ad516-c4f4-11ea-98f7-000d3a38a36f,38.000,46.429,27.000,141.355,1.751,2.501
10489,7af5cd16-b100-11e9-9757-000d3a38a36f,103.143,111.857,43.000,157.113,1.737,2.482
4315,d5ef8058-a5c6-11e9-a2fc-000d3a38a36f,133.143,147.143,49.500,160.203,1.666,2.381
6756,27310582-6362-11ea-a6dc-000d3a38a36f,62.714,64.143,29.000,168.881,1.630,2.328
6666,53fe00d4-7b7a-11eb-960b-000d3a38a36f,9.714,13.000,17.000,259.865,1.622,2.317
10536,e143b6fa-d6f8-11e9-93bc-000d3a38a36f,104.571,113.429,40.000,176.200,1.612,2.303


Gamma-Gamma model

In [101]:
ggf = GammaGammaFitter(penalizer_coef=0.01)

In [102]:
ggf.fit(cltv_df["frequency"], cltv_df["monetary_cltv_avg"])

ValueError: There exist non-integer values in the frequency vector.

In [103]:
cltv_df["frequency"] = cltv_df["frequency"].round().astype(int)

In [104]:
ggf.fit(cltv_df["frequency"], cltv_df["monetary_cltv_avg"])

<lifetimes.GammaGammaFitter: fitted with 19945 subjects, p: 4.15, q: 0.47, v: 4.08>

In [105]:
cltv_df["exp_average_value"] = ggf.conditional_expected_average_profit(cltv_df["frequency"],
                                                                       cltv_df["monetary_cltv_avg"])

In [106]:
cltv_df.head()

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_months,exp_sales_6_months,exp_average_value
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.000,30.571,5,187.874,0.568,0.812,193.633
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.857,224.857,21,95.883,0.574,0.819,96.665
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.286,78.857,5,117.064,0.391,0.559,120.968
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.571,20.857,2,60.985,0.409,0.584,67.320
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.143,95.429,2,104.990,0.231,0.330,114.325


# CLTV

In [107]:
cltv = ggf.customer_lifetime_value(bgf,
                                   cltv_df["frequency"],
                                   cltv_df["recency_cltv_weekly"],
                                   cltv_df["T_weekly"],
                                   cltv_df["monetary_cltv_avg"],
                                   time=6,
                                   freq="M",
                                   discount_rate=0.01)

In [108]:
cltv_df["cltv"] = cltv 

In [109]:
cltv_df.sort_values("cltv", ascending=False)[:10]

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_months,exp_sales_6_months,exp_average_value,cltv
9055,47a642fe-975b-11eb-8c2a-000d3a38a36f,2.857,7.857,4,1401.771,0.638,0.912,1449.030,765.870
13880,7137a5c0-7aad-11ea-8f20-000d3a38a36f,6.143,13.143,11,758.044,1.149,1.642,767.318,730.085
17323,f59053e2-a503-11e9-a2fc-000d3a38a36f,51.714,101.000,7,1106.467,0.421,0.602,1127.611,393.322
12438,625f40a2-5bd2-11ea-98b0-000d3a38a36f,74.286,74.571,16,501.845,0.913,1.304,506.138,382.628
8868,9ce6e520-89b0-11ea-a6e7-000d3a38a36f,3.429,34.429,8,601.226,0.738,1.055,611.493,373.720
7330,a4d534a2-5b1b-11eb-8dbd-000d3a38a36f,62.714,67.286,52,164.633,2.740,3.915,165.116,371.299
6402,851de3b4-8f0c-11eb-8cb8-000d3a38a36f,8.286,9.429,2,862.690,0.463,0.662,923.680,354.167
6666,53fe00d4-7b7a-11eb-960b-000d3a38a36f,9.714,13.000,17,259.865,1.622,2.317,262.073,351.951
19538,55d54d9e-8ac7-11ea-8ec0-000d3a38a36f,52.571,58.714,31,228.530,1.799,2.570,229.607,341.961
14858,031b2954-6d28-11eb-99c4-000d3a38a36f,14.857,15.571,3,743.587,0.508,0.726,778.050,327.503


# Segmentation 

In [110]:
cltv_df["cltv_segment"] = pd.qcut(cltv_df["cltv"], 4,labels= ["D", "C", "B", "A"])

In [111]:
cltv_df.head()

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_months,exp_sales_6_months,exp_average_value,cltv,cltv_segment
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.000,30.571,5,187.874,0.568,0.812,193.633,91.078,A
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.857,224.857,21,95.883,0.574,0.819,96.665,45.899,B
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.286,78.857,5,117.064,0.391,0.559,120.968,39.177,B
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.571,20.857,2,60.985,0.409,0.584,67.320,22.772,D
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.143,95.429,2,104.990,0.231,0.330,114.325,21.867,D
